In [1]:
import json
import mysql.connector

In [2]:
mydb = mysql.connector.connect(
  host="database-1.chgxrutzqyvi.eu-central-1.rds.amazonaws.com",
  user="gmaps",
  passwd="oIc1burIsKZYLRxKjZPN",
  database="innodb"
)
mycursor = mydb.cursor()

In [3]:
# mycursor.execute("SELECT id FROM commercial_premise WHERE name = %s", ("AHOAN",))
# records = mycursor.fetchone()

# Create tables

In [3]:
# Main table
sql_main_table = """
    CREATE TABLE IF NOT EXISTS commercial_premise (
        id INT NOT NULL AUTO_INCREMENT,
        name VARCHAR(100) NOT NULL UNIQUE,
        zip_code INT(5) NOT NULL,
        coordinates VARCHAR(25),
        telephone_number VARCHAR(25),
        opening_hours VARCHAR(100),
        type VARCHAR(20) NOT NULL,
        score FLOAT(2) DEFAULT 0.0,
        total_scores INT(10) DEFAULT 0,
        price_range VARCHAR(5),
        style VARCHAR(20),
        address VARCHAR(40) NOT NULL,
        date DATE NOT NULL,
        PRIMARY KEY(ID)
    )
"""

sql_comments = """
    CREATE TABLE IF NOT EXISTS commercial_premise_comments (
        id INT NOT NULL AUTO_INCREMENT,
        commercial_premise_id INT NOT NULL,
        content VARCHAR(600),
        date DATE NOT NULL,
        PRIMARY KEY(id),
        INDEX prem_ind (commercial_premise_id),
        FOREIGN KEY (commercial_premise_id)
            REFERENCES commercial_premise(id)
            ON DELETE CASCADE
            ON UPDATE CASCADE
    )
"""

sql_ocupation = """
    CREATE TABLE IF NOT EXISTS commercial_premise_occupation (
        id INT NOT NULL AUTO_INCREMENT,
        commercial_premise_id INT NOT NULL,
        week_day VARCHAR(50) NOT NULL,
        time_period CHAR(50) NOT NULL,
        occupation FLOAT DEFAULT 0.0,
        date DATE NOT NULL,
        PRIMARY KEY(id),
        INDEX prem_ind (commercial_premise_id),
        FOREIGN KEY (commercial_premise_id)
            REFERENCES commercial_premise(id)
            ON DELETE CASCADE
            ON UPDATE CASCADE
    )
"""

mycursor.execute("DROP TABLE commercial_premise_occupation")
mycursor.execute("DROP TABLE commercial_premise_comments")
mycursor.execute("DROP TABLE commercial_premise")

mycursor.execute(sql_main_table)
mycursor.execute(sql_comments)
mycursor.execute(sql_ocupation)
mycursor.execute("SHOW TABLES")
mycursor.fetchall()

[('commercial_premise',),
 ('commercial_premise_comments',),
 ('commercial_premise_occupation',)]

In [4]:
from pprint import pprint
sql1 = "DESCRIBE commercial_premise"
sql2 = "DESCRIBE commercial_premise_comments"
sql3 = "DESCRIBE commercial_premise_occupation"
mycursor.execute(sql1)
pprint(mycursor.fetchall())
mycursor.execute(sql2)
pprint(mycursor.fetchall())
mycursor.execute(sql3)
pprint(mycursor.fetchall())

[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('name', 'varchar(100)', 'NO', 'UNI', None, ''),
 ('zip_code', 'int(5)', 'NO', '', None, ''),
 ('coordinates', 'varchar(25)', 'YES', '', None, ''),
 ('telephone_number', 'varchar(25)', 'YES', '', None, ''),
 ('opening_hours', 'varchar(100)', 'YES', '', None, ''),
 ('type', 'varchar(20)', 'NO', '', None, ''),
 ('score', 'float', 'YES', '', '0', ''),
 ('total_scores', 'int(10)', 'YES', '', '0', ''),
 ('price_range', 'varchar(5)', 'YES', '', None, ''),
 ('style', 'varchar(20)', 'YES', '', None, ''),
 ('address', 'varchar(40)', 'NO', '', None, ''),
 ('date', 'date', 'NO', '', None, '')]
[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('commercial_premise_id', 'int(11)', 'NO', 'MUL', None, ''),
 ('content', 'varchar(600)', 'YES', '', None, ''),
 ('date', 'date', 'NO', '', None, '')]
[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('commercial_premise_id', 'int(11)', 'NO', 'MUL', None, ''),
 ('week_day', 'varchar(5

# Read data

In [5]:
with open("../data_48005.json", "r") as file:
    sample = json.loads(file.read())

In [6]:
sample.keys()

dict_keys(['Bar Restaurante Baste', 'Bar Restaurante Víctor', 'Restaurante-Cocktail Yamike', 'Bar Restaurante Bizkaia', 'Café Bar Bilbao', 'Aitxiar', 'La Ribera Bilbao', 'La Olla de la Plaza Nueva', 'Restaurante Kalderapeko', 'con B de bilbao', 'Lontz', 'AHOAN', 'Restaurante Víctor Montes', 'Bar Claudio La Feria Del Jamón', 'Musume Izakaya', 'Restaurante Berton Bukoi', 'Xukela', 'Sorginzulo', 'La Salve', 'Bikandi', 'Bar Rotterdam', 'Tirauki', 'Vizcaya Bi', 'BIKIAK - Bar de Pintxos - BILBAO', 'Casa Víctor Montes', 'Culmen Bilbao', 'Sutan Fire Cuisine & Bar', 'Gure Etxea Taberna', 'El Ciervo', 'Batzoki Bilbo Zaharra', 'Zuga taberna', 'Bar Santa María', 'Los Sandwich de la Mary', 'Huevos Y Tortilla', 'Bertoko Berria', 'Isipil Bar', 'Bar Dartxin', 'Taberna Plaza Nueva', 'Iaela', 'Bar Urdiña', 'Café - Obrador OKAPI', 'BAR RESTAURANTE SENDEJA 5', 'Café Restaurante del Arenal', 'Irrintzi', 'Restaurante Urbieta', 'Bar Bacaicoa', 'Bar Plaza Unamuno', 'Erreka', 'Restaurante BERTON SASIBIL Pintxo

In [7]:
len(sample.keys())

196

# Format data

In [8]:
def decompose_occupancy_data(occupancy_levels):
    occupancy = {
        "lunes": {},
        "martes": {},
        "miercoles": {},
        "jueves": {},
        "viernes": {},
        "sabado": {},
        "domingo": {}
    }
    for week_day, occupancy_levels in occupancy_levels.items():
        if(occupancy_levels is not None):
            for occupancy_level in occupancy_levels:
                if(occupancy_level is not None):
                    try:
                        base = occupancy_level.split(":")[1:]
                        occupancy[week_day].update({
                            base[1].split(")")[0].strip(): float(base[0].split("\xa0%")[0])
                        })
                    except:
                        pass
    return occupancy
        

In [9]:
def write_to_db(element, zip_code, premise_type, mydb, mycursor, date):

    mycursor.execute("SELECT id FROM commercial_premise WHERE name = %s", (element["name"],))
    records = mycursor.fetchone()
    if(records is None):
        #Store element
        if element["address"] is not None:
            compressed_address = [item.strip() for item in element["address"].split("·")]
            if(len(compressed_address) == 3):
                price_range = compressed_address[0]
                style = compressed_address[1]
                address = compressed_address[2]
            else:
                price_range = None
                style = compressed_address[0]
                address = compressed_address[1]
        else:
            price_range = None
            style = None
            address = None
        sql = """
            INSERT INTO commercial_premise 
                (name, zip_code, coordinates, telephone_number, opennig_hours, type, score, total_scores, price_range, style, address, date) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
        keys = element.keys()
        values = (
            element["name"],
            zip_code,
            element["coordinates"] if "coordinates" in keys else None,
            element["telephone_number"] if "telephone_number" in keys else None,
            ",".join(element["opennig_hours"]) if "opennig_hours" in keys and element["opennig_hours"] is not None else None,
            premise_type,
            element["score"],
            element["total_scores"],
            price_range,
            style,
            address,
            date
        )
        mycursor.execute(sql, values)
        mydb.commit()
        element_id = mycursor.lastrowid
    else:
        element_id = records[0]
    #Store comments
    if("comments" in keys):
        sql = """
                INSERT INTO commercial_premise_comments
                (commercial_premise_id, content, date)
                VALUES (%s, %s, %s)
            """
        values = []
        for comment in element["comments"]:
            values.append((element_id, comment, date))
        mycursor.executemany(sql, values)
    #Store occupancy data
    sql = """
        INSERT INTO commercial_premise_occupation
        (
            commercial_premise_id, week_day, time_period, occupation, date
        )
        VALUES (%s, %s, %s, %s, %s)
    """
    if("occupancy" in keys):
        values = []
        for week_day, content in decompose_occupancy_data(element["occupancy"]).items():
            if content is not None and content != {}:
                for key,value in content.items():
                    values.append((element_id, week_day, key, value, date))
        mycursor.executemany(sql, values)
    mydb.commit()

In [10]:
import time

In [12]:
t1 = time.time()
for name, element in sample.items():
    print(name)
    try:
        write_to_db(element, "48005", "restaurante", mydb, mycursor)
    except:
        print("ERROR ", name)
t2 = time.time()
print(t2-t1)

Bar Restaurante Baste
ERROR  Bar Restaurante Baste
Bar Restaurante Víctor
ERROR  Bar Restaurante Víctor
Restaurante-Cocktail Yamike
ERROR  Restaurante-Cocktail Yamike
Bar Restaurante Bizkaia
ERROR  Bar Restaurante Bizkaia
Café Bar Bilbao
Aitxiar
La Ribera Bilbao
La Olla de la Plaza Nueva
Restaurante Kalderapeko
con B de bilbao
Lontz
AHOAN
Restaurante Víctor Montes
Bar Claudio La Feria Del Jamón
Musume Izakaya
Restaurante Berton Bukoi
Xukela
Sorginzulo
La Salve
Bikandi
Bar Rotterdam
Tirauki
Vizcaya Bi
BIKIAK - Bar de Pintxos - BILBAO
Casa Víctor Montes
Culmen Bilbao
Sutan Fire Cuisine & Bar
Gure Etxea Taberna
El Ciervo
Batzoki Bilbo Zaharra
Zuga taberna
Bar Santa María
Los Sandwich de la Mary
Huevos Y Tortilla
Bertoko Berria
Isipil Bar
Bar Dartxin
Taberna Plaza Nueva
Iaela
Bar Urdiña
Café - Obrador OKAPI
BAR RESTAURANTE SENDEJA 5
Café Restaurante del Arenal
Irrintzi
Restaurante Urbieta
Bar Bacaicoa
Bar Plaza Unamuno
Erreka
Restaurante BERTON SASIBIL Pintxos Bilbao
kukusoak taberna
Resta